In [64]:
# # Раскомментировать при необходимости
# !pip install pymorphy2
# !pip install pymorphy2-dicts
# !pip install DAWG-Python

In [65]:
import pymorphy2
import re

In [66]:
morph = pymorphy2.MorphAnalyzer()

In [67]:
# Тестирующая функция №1 (10 тестов)
def test_form_of_word():
    assert not is_form_of_word('игра', 'Игрушки')
    assert is_form_of_word('игрушек', 'Игрушки')
    assert is_form_of_word('Игрушек', 'Игрушки')
    assert is_form_of_word('решать', 'решаю')
    assert is_form_of_word('читал', 'читавшая')
    assert not is_form_of_word('примерчик', 'пример')
    assert is_form_of_word('примеров', 'примерами')
    assert is_form_of_word('она', 'ею')
    assert is_form_of_word('расти', 'растущий')
    assert not is_form_of_word('программировать', 'программами')

In [68]:
# Проверка, является ли одно слово формой другого
def is_form_of_word(form: str, word: str) -> bool: 
    if morph.parse(form)[0][2] == morph.parse(word)[0][2]:
        return True
    else:
        return False

In [69]:
# Тестируем
test_form_of_word()

In [70]:
# Чтение файла (.txt). Возвращает список строк файла
def read_file(path_to_file: str) -> list: 
    list_of_lines = []
    with open(path_to_file, 'r', encoding='utf-8') as file:
        for line in file:
            list_of_lines.append(line)
    return list_of_lines

In [71]:
# Тестирующая функция №2 (10 тестов)
def test_clean_list_of_lines():
    assert clean_list_of_lines([]) == []
    assert clean_list_of_lines(['']) == [[]]
    assert clean_list_of_lines(['', '']) == [[], []]
    assert clean_list_of_lines(['abc', 'bcd*']) == [['abc'], ['bcd']]
    assert clean_list_of_lines(['●text text, text', 'something here ***']) == [['text', 'text', 'text'], ['something', 'here']]
    assert clean_list_of_lines(['Сириус2021']) == [['Сириус2021']]
    assert clean_list_of_lines(['text:\n']) == [['text']]
    assert clean_list_of_lines(['\ttext']) == [['text']]
    assert clean_list_of_lines(['text; text - text, text']) == [['text', 'text', 'text', 'text']]
    assert clean_list_of_lines(['TeXt...']) == [['TeXt']]

In [72]:
# Чистка списка строк файла от символов, которые будем игнорировать. Формат вывода: список списков слов каждой строки
def clean_list_of_lines(list_of_lines: list) -> list: 
    for i in range(len(list_of_lines)):
        list_of_lines[i] = list_of_lines[i].replace('\t', '')
        list_of_lines[i] = list_of_lines[i].replace('\r', '')
        list_of_lines[i] = list_of_lines[i].replace('\n', '')
        for symbol in list_of_lines[i]:
            if not symbol.isalnum() and not symbol == ' ':
                list_of_lines[i] = list_of_lines[i].replace(symbol, '')
        list_of_lines[i] = re.sub(" +", " ", list_of_lines[i])
        list_of_lines[i] = list_of_lines[i].split()
    return list_of_lines 

In [73]:
# Тестируем
test_clean_list_of_lines()

In [74]:
# Поиск контекста слова и его словоформ с возможностью вывода статистики, морфологического анализа, а также фильтрации
# по морфологическим признакам
def find_сontext_for_one_word(word: str,  # Список слов, словоформы которых будут найдены (list)
                 all_words: list,  # Массив всех слов в тексте по порядку
                 list_of_lines: list,  # Путь до текстового файла (.txt) (str)
                 statistics=True,  # Выводить статистику по количеству найденных слов и словоформ (bool)
                 morphological_analysis=True,  # Выводить морфологический анализ слов (bool)
                 filter_by_morphological_features=False  # Выводить слова, морфологические признаки которых содержат
                                                         # все из перечиселенных в списке признаков.
                                                         # Иначе, False (list or bool)
                ):
    count = 0
    count_exactly_word = 0
    count_forms_of_word = 0
    for i in range(len(list_of_lines)):
        for w in range(len(list_of_lines[i])):
            if is_form_of_word(all_words[count], word): # Проверка, является ли слово словоформой исходного слова
                if (type(filter_by_morphological_features) == list and \
                len(set(str(morph.parse(all_words[count])[0][1]).split(',')) & \
                set(filter_by_morphological_features)) == len(set(filter_by_morphological_features))) \
                or type(filter_by_morphological_features) == bool:  # Проверка на фильтры
                    if all_words[count].lower() == word.lower():
                        count_exactly_word += 1
                    else:
                        count_forms_of_word += 1
                    to_print = []
                    if count >= 2:
                        to_print.append(all_words[count - 2])
                    if count >= 1:
                        to_print.append(all_words[count - 1])
                    to_print.append(all_words[count])
                    if count <= len(all_words) - 2:
                        to_print.append(all_words[count + 1])
                    if count <= len(all_words) - 3:
                        to_print.append(all_words[count + 2])
                    print(f'>> Номер строки: {i + 1}. Контекст: ' + ' '.join(to_print))
                    if morphological_analysis:
                        print('Морфологические признаки: ' + ', '.join(str(morph.parse(all_words[count])[0][1]).split(',')))
                    print('----------------------------------------------')
            count += 1
    if statistics:
        print(f'Найдено {count_exactly_word} слов "{word}" и {count_forms_of_word} словоформ')
    print('______________________________________________')

In [75]:
# Та же функция, но с чтением файла и возможностью поиска нескольких слов
def find_сontext(words: list,  # Список слов, словоформы которых будут найдены (list)
                 path_to_file: str,  # Путь до текстового файла (.txt) (str)
                 statistics=True,  # Выводить статистику по количеству найденных слов и словоформ (bool)
                 morphological_analysis=True,  # Выводить морфологический анализ слов (bool)
                 filter_by_morphological_features=False  # Выводить слова, морфологические признаки которых содержат
                                                         # все из перечиселенных в списке признаков.
                                                         # Иначе, False (list or bool)
                ):
    list_of_lines = clean_list_of_lines(read_file(path_to_file))
    all_words = []
    for i in range(len(list_of_lines)):
        for w in range(len(list_of_lines[i])):
            all_words.append(list_of_lines[i][w])
    for word in words:
        find_сontext_for_one_word(word, all_words, list_of_lines, statistics, 
                                  morphological_analysis, filter_by_morphological_features)

### Доступные морфологические признаки можно посмотреть здесь:
https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs

In [76]:
# Пример использования для нескольких слов
find_сontext(['файл', 'формат'],
              'tests/test1.txt', 
               statistics=True,
               morphological_analysis=True,
               filter_by_morphological_features=False)

>> Номер строки: 6. Контекст: либо из файлов либо из
Морфологические признаки: NOUN, inan, masc plur, gent
----------------------------------------------
>> Номер строки: 6. Контекст: сохранять в файлы Форматы входных
Морфологические признаки: NOUN, inan, masc plur, accs
----------------------------------------------
>> Номер строки: 7. Контекст: содержать примеры файлов соответствующих форматов
Морфологические признаки: NOUN, inan, masc plur, gent
----------------------------------------------
>> Номер строки: 11. Контекст: сопровождаться документацией файл READMEmd в
Морфологические признаки: NOUN, inan, masc sing, nomn
----------------------------------------------
>> Номер строки: 27. Контекст: Дан текстовый файл содержащий текст
Морфологические признаки: NOUN, inan, masc sing, nomn
----------------------------------------------
>> Номер строки: 27. Контекст: строки в файле игнорируя знаки
Морфологические признаки: NOUN, inan, masc sing, loct
---------------------------------------

In [77]:
# Пример использования для одного слова с фильтрацией по морфологическим признакам
find_сontext(['душа'],
              'tests/test2.txt', 
               statistics=True,
               morphological_analysis=True,
               filter_by_morphological_features=['NOUN', 'nomn'])

>> Номер строки: 38. Контекст: Была бы душа жива Украдкой
Морфологические признаки: NOUN, inan, femn sing, nomn
----------------------------------------------
Найдено 1 слов "душа" и 0 словоформ
______________________________________________


In [78]:
# Пример использования без морфологического анализа
find_сontext(['каждый', 'другой','всякий'],
              'tests/test2.txt', 
               statistics=True,
               morphological_analysis=False,
               filter_by_morphological_features=False)

>> Номер строки: 13. Контекст: железом У каждого сад и
----------------------------------------------
>> Номер строки: 14. Контекст: гумно У каждого крашены ставни
----------------------------------------------
>> Номер строки: 318. Контекст: стеречь У каждого хата гнилая
----------------------------------------------
>> Номер строки: 349. Контекст: других И каждый с улыбкой
----------------------------------------------
Найдено 1 слов "каждый" и 3 словоформ
______________________________________________
>> Номер строки: 89. Контекст: рев мортир Другую явил я
----------------------------------------------
>> Номер строки: 298. Контекст: до обеда Другое сдержу про
----------------------------------------------
>> Номер строки: 347. Контекст: себя и других И каждый
----------------------------------------------
>> Номер строки: 421. Контекст: Довольно Найдемте другой язык Но
----------------------------------------------
>> Номер строки: 688. Контекст: перевел на другое Уставясь в
------